In [1]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO


In [2]:
import pickle

data_file = open("data.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("parameters.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface()

In [3]:
param


{'probs': tensor([0.2000, 0.6000, 0.0500, 0.0250, 0.0250]),
 'init_probs': 0.6,
 'purity': 0.48,
 'theta_shape_atac': tensor([12.4963, 16.6187,  4.3866,  9.7522]),
 'theta_rate_atac': tensor([0.0135, 0.0067, 0.0125, 0.0125]),
 'nb_size_init_atac': tensor([150., 150., 150., 150.]),
 'likelihood_atac': 'NB',
 'a': 0.1,
 'b': 100.0,
 'lambda': 0.0,
 'hidden_dim': 5,
 'binom_prior_limits': [40.0, 1000.0],
 'equal_sizes_sd': True,
 'Temperature': 20.0,
 'K': [1, 2, 3],
 'latent_type': 'G'}

In [4]:
param["binom_prior_limits"] = torch.tensor([10,1000])
param["K"] = 2
param["likelihood_atac"] = "NB"
param["temperature"] = 20
param["lambda"] = 0


In [5]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(TraceGraph_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)


In [6]:
ll = interface.run(steps = 2000, param_optimizer = {"lr":0.01},  )



Running LatentCategorical on 2534 cells with 4 segments for 2000 steps and 1 modalities


ELBO: 6.022332605  : 100%|██████████| 2000/2000 [00:12<00:00, 156.42it/s]


Done!


In [7]:

import numpy as np


lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.


In [8]:
lr

{'mixture_weights_atac': array([0.4439953 , 0.55600476], dtype=float32),
 'NB_size_atac': array([17.54844 , 16.8888  , 15.924793, 35.700268], dtype=float32),
 'segment_factor_atac': array([ 535.528  , 1230.6976 ,  359.27823,  836.60657], dtype=float32),
 'CNV_probabilities': array([[[3.31589510e-03, 9.93974388e-01, 1.97842950e-03, 3.55056749e-04,
          3.76298442e-04],
         [1.30827630e-05, 9.99980927e-01, 2.91865240e-06, 1.33876279e-06,
          1.86226157e-06],
         [2.83223599e-01, 7.16249526e-01, 1.91705170e-04, 1.74035260e-04,
          1.61211778e-04],
         [3.75419222e-06, 9.99995470e-01, 2.07485314e-07, 5.20058393e-07,
          6.05457728e-08]],
 
        [[6.10584438e-01, 3.88696402e-01, 3.28973372e-04, 2.03493764e-04,
          1.86731035e-04],
         [1.14857512e-05, 9.99988198e-01, 1.75076991e-07, 1.08175477e-07,
          1.11891119e-07],
         [5.71394985e-08, 1.84721586e-07, 3.35060477e-08, 9.99895453e-01,
          1.04309045e-04],
         [5.116

In [24]:

data["data_atac"][0,:].mean()

tensor(335.6740, dtype=torch.float64)

In [43]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [46]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

tensor([[[9.9939e-01, 9.9989e-01, 1.4069e-04,  ..., 6.5166e-03,
          5.3351e-01, 1.5664e-07],
         [9.3591e-01, 1.6383e-01, 1.8467e-02,  ..., 6.9422e-01,
          8.2800e-01, 1.4531e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         ...,
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01],
         [5.0354e-01, 5.0354e-01, 5.0354e-01,  ..., 5.0354e-01,
          5.0354e-01, 5.0354e-01]],

        [[6.1418e-04, 1.1402e-04, 9.9986e-01,  ..., 9.9348e-01,
          4.6649e-01, 1.0000e+00],
         [6.4089e-02, 8.3617e-01, 9.8153e-01,  ..., 3.0578e-01,
          1.7200e-01, 8.5469e-01],
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.9646e-01, 4.9646e-01],
         ...,
         [4.9646e-01, 4.9646e-01, 4.9646e-01,  ..., 4.9646e-01,
          4.964

In [10]:
np.sum(lr["assignment_atac"] == 0)

1209

In [11]:
data

{'data_atac': tensor([[ 235.,  406.,  390.,  ...,  384.,  668.,  366.],
         [ 804.,  726., 1380.,  ..., 1089., 1721., 1406.],
         [ 378.,  334.,  773.,  ...,  217.,  253.,  152.],
         [ 966., 1204., 1999.,  ...,  569.,  718.,  851.]], dtype=torch.float64),
 'norm_factor_atac': tensor([0.3387, 0.3357, 0.6044,  ..., 0.3929, 0.6118, 0.3815],
        dtype=torch.float64),
 'pld': tensor([3., 3., 1., 1.], dtype=torch.float64),
 'segments': 2}